# Performance Evaluation -- Reductions

In this notebook, we report on some experimental results obtained with Kong.

### Setup

Import librairies.

In [ ]:
%matplotlib inline

import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

Path to data.

In [ ]:
path_data = '../../csv/'
path_lists = '../../instances/MCC2021_lists/'

## Load Data Frames

### All instances.

In [ ]:
# Read 'reductions.csv'
df_reductions = pd.read_csv(path_data + 'reductions.csv')
df_reductions

### Safe instances.

In [ ]:
# Read safe nets
df_safe_nets = pd.read_csv(path_lists + 'safe_list', names=["INSTANCE"], header=None)
df_reductions_safe = df_reductions.merge(df_safe_nets, how='inner', on='INSTANCE')
df_reductions_safe

## Analysis

In [ ]:
def draw_figure(df, name, xlim):
    """ Compare the reduction ratio between the full reduce and the TFGs.
    """
    fontsize = 23

    ratio_frequency_reduce = df['RATIO_REDUCE'].value_counts().sort_index(ascending=False)
    df_ratio_frequency_reduce = pd.DataFrame({'INSTANCES': ratio_frequency_reduce.values}).cumsum()
    df_ratio_frequency_reduce['RATIO_REDUCE'] = ratio_frequency_reduce.index

    df_first_row_reduce = pd.DataFrame([{'INSTANCES': 1, 'RATIO_REDUCE': df_ratio_frequency_reduce['RATIO_REDUCE'][0]}])
    df_ratio_frequency_reduce = pd.concat([df_first_row_reduce, df_ratio_frequency_reduce])
    ax = df_ratio_frequency_reduce.plot.area(x='INSTANCES', xlim=(0,df_reductions.shape[0]), ylim=(0,100), figsize=(12,10))
    ax.legend(["Best possible reduction with Reduce", "Reduction leading to a well-formed TFG"])

    # Count instances with the same reduction ratio
    ratio_frequency = df_reductions['RATIO_TFG'].value_counts().sort_index(ascending=False)

    # Cumulative data frame
    df_ratio_frequency = pd.DataFrame({'INSTANCES': ratio_frequency.values}).cumsum()
    df_ratio_frequency['RATIO_TFG'] = ratio_frequency.index

    # Add row '0'
    df_first_row = pd.DataFrame([{'INSTANCES': 1, 'RATIO_TFG': df_ratio_frequency['RATIO_TFG'][0]}])
    df_ratio_frequency = pd.concat([df_first_row, df_ratio_frequency])

    # Draw instances reduction ratio
    df_ratio_frequency.plot.area(x='INSTANCES', xlim=(0,df_reductions.shape[0]), ylim=(0,100), legend='', ax=ax)
    plt.xlabel('Number of instances', fontsize=fontsize)
    plt.ylabel('Reduction ratio (%)', fontsize=fontsize)
    L = plt.legend(fontsize=fontsize)
    L.get_texts()[0].set_text('Best possible reduction with Reduce')
    L.get_texts()[1].set_text('Reduction leading to a well-formed TFG')
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.xlim(xlim)
    plt.grid()
    plt.savefig('pics/{}.png'.format(name), bbox_inches = 'tight')
    plt.show()

In [ ]:
def table(df):
    """ Summary table.
    """
    def table_helper(df, kind):
        return [kind, df['TIME_' + kind].median(), df['TIME_' + kind].mean(), df['RATIO_' + kind].median(), df['RATIO_' + kind].mean()]

    performance_overview = pd.DataFrame([table_helper(df, kind) for kind in ['TFG', 'REDUCE']], columns=['Kind', 'Median Reduction Ratio', 'Mean Reduction Ratio', 'Median Reduction Time', 'Mean Reduction Time'])
    performance_overview.set_index('Kind')
    return performance_overview

### All instances.

In [ ]:
table(df_reductions)

In [ ]:
draw_figure(df_reductions, "reductions_all_instances", [0, 1100])

### Safe instances.

In [ ]:
table(df_reductions_safe)

In [ ]:
draw_figure(df_reductions_safe, "reductions_safe_instances", [0, 600])